In [2]:
import data_loader
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import matplotlib.pyplot as plt
import sklearn.linear_model as linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# np.random.seed(0)
# np.__version__

In [3]:
column_names = [
     'age', 'workclass', 'fnlwgt', 'education', 'education-num', 
     'marital-status', 'occupation', 'relationship', 'race', 'sex', 
     'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
# df = pd.read_csv("data/adult.data", header=None, names=column_names)
# print(df.shape)
# df.sample(n=3)

In [4]:
train, validation = data_loader.load_train_data('data/adult.data')
train['train'] = 1
train['test'] = 0
# train.sample(n=3)

In [5]:
validation['train'] = 0
validation['test'] = 0
# validation.sample(n=3)

In [6]:
test = data_loader.load_test_data('data/adult.test')
test['train'] = 0
test['test'] = 1
# test.sample(n=3)

In [7]:
df_combined = pd.concat([train, validation, test])
df_combined.columns = column_names + ['train','test']
df_combined.sample(n=3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,train,test
17133,31,Private,176410,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,38,United-States,<=50K,1,0
26123,40,Private,83953,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,50,United-States,>50K,1,0
961,32,Private,169240,HS-grad,9,Divorced,Machine-op-inspct,Not-in-family,White,Female,0,0,38,United-States,<=50K,1,0


In [8]:
for c in df_combined.columns:
    df_combined[c] = df_combined[c].replace(' ?', np.nan)

df_combined['salary'] = df_combined['salary'].replace(' <=50K.', ' <=50K')
df_combined['salary'] = df_combined['salary'].replace(' >50K.', ' >50K')
df_combined.dropna(how='any',inplace=True)

In [9]:
df_combined['salary'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [10]:
df_encoded = pd.get_dummies(df_combined)

for i in df_encoded.columns:
    print(i)

age
fnlwgt
education-num
capital-gain
capital-loss
hours-per-week
train
test
workclass_ Federal-gov
workclass_ Local-gov
workclass_ Private
workclass_ Self-emp-inc
workclass_ Self-emp-not-inc
workclass_ State-gov
workclass_ Without-pay
education_ 10th
education_ 11th
education_ 12th
education_ 1st-4th
education_ 5th-6th
education_ 7th-8th
education_ 9th
education_ Assoc-acdm
education_ Assoc-voc
education_ Bachelors
education_ Doctorate
education_ HS-grad
education_ Masters
education_ Preschool
education_ Prof-school
education_ Some-college
marital-status_ Divorced
marital-status_ Married-AF-spouse
marital-status_ Married-civ-spouse
marital-status_ Married-spouse-absent
marital-status_ Never-married
marital-status_ Separated
marital-status_ Widowed
occupation_ Adm-clerical
occupation_ Armed-Forces
occupation_ Craft-repair
occupation_ Exec-managerial
occupation_ Farming-fishing
occupation_ Handlers-cleaners
occupation_ Machine-op-inspct
occupation_ Other-service
occupation_ Priv-house-s

In [11]:
df_encoded.to_csv('data/combined_adult.csv')

In [12]:
df_train = df_encoded[(df_encoded['train']==1) & (df_encoded['test']==0)].copy()
df_valid = df_encoded[(df_encoded['train']==0) & (df_encoded['test']==0)].copy()
df_test  = df_encoded[(df_encoded['train']==0) & (df_encoded['test']==1)].copy()
df_train.drop(["train", "test"], axis=1, inplace=True)
df_valid.drop(["train", "test"], axis=1, inplace=True)
df_test.drop(["train", "test"], axis=1, inplace=True)

In [24]:
df_tr = df_train.columns.values
df_va = df_valid.columns.values
df_te = df_test.columns.values
for i, v in enumerate(df_tr):
    print('feature{:d}: train: {:>25}, validation: {:>25}, test: {:>25}'.format(i, v, df_va[i], df_te[i]))

feature0: train:                       age, validation:                       age, test:                       age
feature1: train:                    fnlwgt, validation:                    fnlwgt, test:                    fnlwgt
feature2: train:             education-num, validation:             education-num, test:             education-num
feature3: train:              capital-gain, validation:              capital-gain, test:              capital-gain
feature4: train:              capital-loss, validation:              capital-loss, test:              capital-loss
feature5: train:            hours-per-week, validation:            hours-per-week, test:            hours-per-week
feature6: train:    workclass_ Federal-gov, validation:    workclass_ Federal-gov, test:    workclass_ Federal-gov
feature7: train:      workclass_ Local-gov, validation:      workclass_ Local-gov, test:      workclass_ Local-gov
feature8: train:        workclass_ Private, validation:        workclass_ Privat

In [14]:
# for i in range(train.shape[1]-1):
#     print('Feature {:2d} count  -  Training: {:5d}, Validation: {:5d}, Test: {:5d}'.format(i, train[i].nunique(), val[i].nunique(), test[i].nunique()))